The purpose of this notebook is to join the disparate tables into one dataset

In [17]:
import pandas as pd
import numpy as np

# launch a qtconsole if first time through
try:
    qtc
except NameError:
    %qtconsole
finally:
    qtc = True

In [22]:
# teacher ratings per school ----------------------------------------------------------------------------
# data on teacher ratings
school_researcher = pd.read_csv('../data/table_export/APPR_SCHOOL_RESEARCHER_FILE_DATA.csv')
school_researcher = school_researcher.loc[school_researcher.DISTRICT_NAME.str.contains('NEW YORK CITY'), :]
school_researcher.OVERALL_SCORE = school_researcher.OVERALL_SCORE.convert_objects(convert_numeric=True)
school_researcher.dropna(inplace=True)

# unique school ids
unique_ids = school_researcher.SCHOOL_BEDS.unique()

# average teacher rating per school
avg_teacher_rating = school_researcher[['SCHOOL_BEDS', 'OVERALL_SCORE']].groupby('SCHOOL_BEDS').mean()
avg_teacher_rating.reset_index(inplace=True)
avg_teacher_rating.columns = ['ENTITY_CD', 'AVG_TEACHER_RATING']

# start piecing together our dataset
consolidated_data = pd.DataFrame({'ENTITY_CD': unique_ids})
consolidated_data = pd.merge(consolidated_data, avg_teacher_rating, on='ENTITY_CD')

# match unique_ids to school reacher info
colsToKeep = ['DISTRICT_BEDS', 'DISTRICT_NAME', 'SCHOOL_BEDS', 'SCHOOL_NAME']
school_researcher = school_researcher.loc[:, colsToKeep].drop_duplicates()
school_researcher.columns = [word.replace('SCHOOL_BEDS', 'ENTITY_CD') for word in school_researcher.columns]
consolidated_data = pd.merge(consolidated_data, school_researcher, on='ENTITY_CD')


# avg school academic performance -----------------------------------------------------------------------
# Now get test score data
regents_cc = pd.read_csv('../data/table_export/Regents_Common_Core_Examination_Annual_Results.csv')

# filter out schools not in our data
f = [i in unique_ids for i in regents_cc.ENTITY_CD.values]
regents_cc = regents_cc.loc[f, :]

# Now lets just look at the 'All Students' subgroup since 
# we're attempting to get a feel for school performance
regents_cc = regents_cc.loc[regents_cc.SUBGROUP_NAME == 'All Students', :]

# Almost all schools have a result for 'REG_COMALG1'.
regents_cc = regents_cc.loc[regents_cc.SUBJECT == 'REG_COMALG1', :]

# There are a few schools with a score of 's'. Let's drop these:
regents_cc = regents_cc.loc[regents_cc.PER_LEVEL1 != 's', :]

# calculate average per school
per_cols = ['PER_LEVEL1', 'PER_LEVEL2', 'PER_LEVEL3', 'PER_LEVEL4', 'PER_LEVEL5']
overall = np.array([0.]*regents_cc.shape[0])
for i, c in enumerate(per_cols):
    overall += (i+1) * (regents_cc[c].values.astype(float) / 100)
regents_cc['AVG_TEST_LEVEL'] = overall

# cut this down to the information we're interested in
colsToKeep = ['ENTITY_CD', 'TESTED', 'AVG_TEST_LEVEL']
regents_cc = regents_cc.loc[:, colsToKeep]

# we dont' have test data for all the schools in consolidated_date
# we need to filter before we join
f = [i in regents_cc.ENTITY_CD.values for i in consolidated_data.ENTITY_CD.values]
consolidated_data = consolidated_data.loc[f, :]

# create a forced cluster column
fc = regents_cc.AVG_TEST_LEVEL > 2.75
fc = fc.astype('int')
regents_cc['FORCED_CLUSTER'] = fc

# now merge test score info into our dataset
consolidated_data = pd.merge(consolidated_data, regents_cc, on = 'ENTITY_CD')


# demographic information ------------------------------------------------------------------------------
demographics = pd.read_csv('../data/table_export/Demographic_Factors.csv')

# Let's just use information from 2014
demographics = demographics.loc[demographics.YEAR == 2014, :]

# filter to the schools in consolidated_data
f = [i in consolidated_data.ENTITY_CD.values for i in demographics.ENTITY_CD.values]
demographics = demographics.loc[f, :]

# drop data we already have
dropCols = ['ENTITY_NAME', 'YEAR']
colsToKeep = [c not in dropCols for c in demographics.columns]
demographics = demographics.loc[:, colsToKeep]

# merge into consolidated_data
consolidated_data = pd.merge(consolidated_data, demographics, on = 'ENTITY_CD')


# staff info ------------------------------------------------------------------------------------
staff = pd.read_csv('../data/table_export/Staff.csv')

# filter on latest year
staff = staff.loc[staff.YEAR == 2014, :]

# keep information for entries in consolidated_data
f = [i in consolidated_data.ENTITY_CD.values for i in staff.ENTITY_CD.values]
staff = staff.loc[f, :]

# drop data we already have
dropCols = ['SCHOOL_NAME', 'YEAR', 'DISTRICT_NAME']
colsToKeep = [c not in dropCols for c in staff.columns]
staff = staff.loc[:, colsToKeep]

# merge into consolidated_data
consolidated_data = pd.merge(consolidated_data, staff, on = 'ENTITY_CD')


# export to csv ---------------------------------------------------------------------------

# export this to a csv file
consolidated_data.to_csv('../data/consolidated_data.csv')

In [23]:
for n in consolidated_data.columns:
    print n

ENTITY_CD
AVG_TEACHER_RATING
DISTRICT_BEDS
DISTRICT_NAME
SCHOOL_NAME
TESTED
AVG_TEST_LEVEL
FORCED_CLUSTER
NUM_FREE_LUNCH
PER_FREE_LUNCH
NUM_REDUCED_LUNCH
PER_REDUCED_LUNCH
NUM_LEP
PER_LEP
NUM_AM_IND
PER_AM_IND
NUM_BLACK
PER_BLACK
NUM_HISP
PER_HISP
NUM_ASIAN
PER_ASIAN
NUM_WHITE
PER_WHITE
NUM_Multi
PER_Multi
NUM_SWD
PER_SWD
NUM_FEMALE
PER_FEMALE
NUM_MALE
PER_MALE
NUM_ECDIS
PER_ECDIS
DISTRICT_CD
CSO_NAME
STREET
CITY
PHONE
GRADE_RANGE
NUM_TEACH
NUM_NO_VALID_CERT
PER_NO_VALID_CERT
NUM_TEACH_OUT_CERT
PER_TEACH_OUT_CERT
NUM_FEWER_3YRS_EXP
PER_FEWER_3YRS_EXP
NUM_MAS_PLUS
PER_MAS_PLUS
NUM_CORE
PER_NOT_HQ
NUM_CLASS
NUM_NO_APPROP_CERT
PER_NO_APPROP_CERT
PER_TURN_FIVE_YRS
PER_TURN_ALL
NUM_FT_OTH_PROF_STAFF
NUM_PT_OTH_PROF_STAFF
NUM_FT_PARAPROFESSIONALS
NUM_PT_PARAPROFESSIONALS
NUM_FT_ASST_PRINCIPALS
NUM_PT_ASST_PRINCIPALS
NUM_FT_PRINCIPALS
NUM_PT_PRINCIPALS
